*Updated for Feb 2026*

Thank you for your interest in this work! This is going to be a more technical look at how I classified Jeopardy categories and made the points I made in the article. At this point this code is up to five years old in some parts and may not reflect latest-and-greatest updates in tech.

**Category Labeling Process**

1. Initial data imports, package imports, and first cleaning steps
2. Pre-processing for NLP-based clustering
3. Running the clustering
4. Consolidating multiple runs of cluster labels
5. Linear regression over cluster tags and final labeling
6. Data analysis and conclusions used in final article

# Initial Setup, Data Imports, Cleaning, Etc.

**Package import statements**

In [21]:
#I don't need all of these but I don't much feel like spending the time to check all the code to confirm the absence

import pandas as pd #dataframe package
import os           #for navigating xml library
import json         #for navigating json
#import ijson        #same
from pandas import json_normalize #same
import numpy as np  #habit
import itertools
import random
import ast
from collections import Counter
from collections import OrderedDict
import collections
import ast
import time
import re

import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

import csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression, Ridge
from itertools import chain

import scipy.cluster.hierarchy as shc

from pandasql import sqldf

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import itertools
import string

import random
from random import shuffle

from nltk.corpus import stopwords

**Stopwords for NLP**

This is a combination of:

* A base list of stopwords I found somewhere on the internet, sorry for poor sourcing
* Words and phrases like "Clue Crew" and "Jeopardy!" that interfered with the clustering process
* Words and half-words like "u" and "s" that popped up when I was making flashcards that weren't providing useful information

In [22]:
stopwords_list = ["u", "s", ":", "this", "'s", ",", "'", ".", "i",\
                  "me", "my", "myself", "we", "our", "ours", "ourselves",\
                  "you", "your", "yours", "yourself", "yourselves", "he",\
                  "him", "his", "himself", "she", "her", "hers", "herself","also",\
                  "it", "its", "itself", "they", "them", "their", "theirs", "themselves",\
                  "what", "which", "who", "whom", "this", "that", "these", "those", "am",\
                  "is", "are", "was", "were", "be", "been", "being", "have", "has", "had",\
                  "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if",\
                  "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",\
                  "against", "between", "into", "through", "during", "before", "after", "above", "below",\
                  "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further",\
                  "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",\
                  "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",\
                  "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now",';',\
                  '(',')','1','2','3','4','5','6','7','8','9','10','#','!','&','``',"''",'\\','wa',"n't",\
                  'clue', 'crew', 'jimmy', 'sarah', 'jeopardy', 'jeopardy!', 'nt', 'trebek', \
                  '__', '___', '____', '_____', '______', '_______', '________', '_________', 'blank', 'wa',\
                  'ha', 'one', 'like', 'name', 'called', 'first', '...', '?', "'re", "might", "got", "said", #new 2024-25
                  "could", "type", "made", "'ll", "'re", "'ve", "got", "get", "say" , "*", \
                  'll', 're', 've'] #new 2026


tags = ['art'
,'business'
,'civics'
,'classical music'
,'fashion'
,'film'
,'food'
,'geography'
,'history'
,'literature'
,'miscellaneous'
,'pop culture'
,'pop music'
,'religion'
,'science'
,'sports'
,'television'
,'theater']


**Dataset Imports**

Updated version of the clue dataset available here: https://github.com/jwolle1/jeopardy_clue_dataset

In [3]:
j = pd.read_csv('combined_season1-41.tsv', sep='\t', header=0)

#the j stands for jeopardy

C:\Users\zachg\AppData\Local\Temp\ipykernel_48020\1582720217.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  j = pd.read_csv('combined_season1-41.tsv', sep='\t', header=0)


**Lemmatization, Cleaning, Initial Prep**

This only needs to be run once and saved.

In [ ]:
lemmatizer = WordNetLemmatizer()

# episodes 4573 (jun 23 2004) and 6420 (jul 13 2012) contain clues asking to finish the agatha christie novel "and then there were...",
# which messes up python, this fixes that for now, but row numbers will change in future files so you would need to search by date to make the 
# fixes below

## season 40 file

#j.loc[247505, 'question'] = 'None'
#j.loc[355714, 'question'] = 'None'

## season 41 file

j.loc[247676, 'question'] = 'None'
j.loc[355885, 'question'] = 'None'

# cleaning questions and answers by lemmatizing them, converting them to lowercase, and removing stopwords

question_mod = [word_tokenize(item) for item in j.question]

question_mod_2 = list()

for item in question_mod:
    item = [lemmatizer.lemmatize(word) for word in item]
    question_mod_2.append(item)
    
question_mod_3 = list()

for item in question_mod_2:
    item = [x.lower() for x in item if x.lower() not in stopwords_list]
    question_mod_3.append(item)

j['question_modified'] = [' '.join(item) for item in question_mod_3]
j['question_modified'] = [item.translate(str.maketrans('', '', string.punctuation)) for item in j.question_modified]
j['question_modified'] = [item.strip() for item in j.question_modified]

answer_mod = [word_tokenize(item) for item in j.answer]

answer_mod_2 = list()

for item in answer_mod:
    item = [lemmatizer.lemmatize(word) for word in item]
    answer_mod_2.append(item)
    
answer_mod_3 = list()

for item in answer_mod_2:
    item = [x.lower() for x in item if x.lower() not in stopwords_list]
    answer_mod_3.append(item)
    
j['answer_modified'] = answer_mod_3

j.to_pickle('s41_cleaned.pkl')

**Reviewing cleaned output**

You can skip here if you've run the above.

In [5]:
j = pd.read_pickle('s41_cleaned.pkl')

In [8]:
#current output

j[j.air_date == '2022-02-03'][['clue_value', 'category', 'air_date', 'question_modified', 'answer_modified']].head()

,clue_value,category,air_date,question_modified,answer_modified
481149,200,THE 20th CENTURY,2022-02-03,princess diana,"[death, following, car, crash, pont, de, l'alm..."
481150,400,THE 20th CENTURY,2022-02-03,decathlon pentathlon,"[olympic, game, stockholm, jim, thorpe, won, g..."
481151,800,THE 20th CENTURY,2022-02-03,operation desert storm,"[invasion, kuwait, prompted, allied, operation..."
481152,1000,THE 20th CENTURY,2022-02-03,fine gael,"[1948, john, costello, became, irish, prime, m..."
481153,200,IT'S A NATIONAL THING,2022-02-03,chinese checker,"[deriving, game, halma—greek, jump, —this, boa..."


# Preparing the Categories for Clustering by Content

With code lifted and adapted from: https://avidml.wordpress.com/2016/10/18/sample-code-to-cluster-unstructured-text/

In [9]:
def to_comma_list(string):
    li = list(string.split(" "))
    return li

def combine_lists(list1, list2):
    lis = list1 + list2
    return(lis)

In [12]:
# stripping question, category and answer text down to a list of the non-stopwords used in the clues
# variables are the presence or absence of a word; i do not weight for multiple uses of a word

j['question_modified'] = [str(i) for i in j.question_modified]
j['question_list'] = [to_comma_list(i) for i in j.question_modified]

j['answer_list'] = [[k.strip() for k in i] for i in j.answer_modified]

j['cat_list'] = [to_comma_list(i.lower()) for i in j.category]
j['cat_list'] = [[lemmatizer.lemmatize(word) for word in lis] for lis in j.cat_list]

j['all_term_list'] = j['question_list'] + j['answer_list'] + j['cat_list']
j['all_term_list'] = [[i for i in x if i not in stopwords_list] for x in j.all_term_list]

In [16]:
# one row of data through this step

for i, k in zip(j.columns, j.loc[481047]):
    print(str(i) + ': ' + str(k))

round: 1
clue_value: 400
daily_double_value: 0
category: 4-LETTER GEOGRAPHY
comments: nan
answer: A natural history museum, the Koenig is in this city, once the capital of West Germany
question: Bonn
air_date: 2022-02-01
notes: nan
question_modified: bonn
answer_modified: ['natural', 'history', 'museum', 'koenig', 'city', 'capital', 'west', 'germany']
question_list: ['bonn']
answer_list: ['natural', 'history', 'museum', 'koenig', 'city', 'capital', 'west', 'germany']
cat_list: ['4-letter', 'geography']
all_term_list: ['bonn', 'natural', 'history', 'museum', 'koenig', 'city', 'capital', 'west', 'germany', '4-letter', 'geography']


In [14]:
k = j[['category', 'all_term_list']] #the k stands for keopardy
for_cats_gp = k.groupby('category', as_index=False).sum()
for_cats_gp['all_term_list_deduped'] = [list(set(x)) for x in for_cats_gp.all_term_list] #removes duplicates

for_cats_gp['all_term_list2'] = [" ".join(x) for x in for_cats_gp.all_term_list_deduped]
for_cats_gp['len'] = for_cats_gp.all_term_list2.str.len()
for_cats_std = for_cats_gp.sort_values('len', ascending=False)

In [17]:
# example output - "all term list2", a list of all key words tied to a category, will become the input for the 
# category clustering that's coming up in the next step

for i, k in zip(for_cats_std.columns, for_cats_std.loc[43196]):
    print(str(i) + ': ' + str(k))
    print(' ')

category: THE DAY
 
all_term_list: ['april', 'fools', 'day', 'thing', 'reported', 'day', 'flying', 'penguin', 'google', 'translate', 'animal', 'tree', 'grow', 'spaghetti', 'day', 'ihop', '2024', 'restaurant', 'chain', 'partnered', 'feeding', 'america', 'feed', 'america', 'national', 'pancake', 'day', 'day', 'juneteenth', '2022', 'new', 'yorker', 'honored', 'newish', 'federal', 'holiday', 'cover', 'painting', 'black', 'family', '157', 'years', 'day', 'cyber', 'monday', '2005', 'shop.org', 'press', 'release', 'public', 'use', 'term', 'shopping', 'day', 'day', 'thanksgiving', 'day', 'indigenous', 'peoples', 'day', '2021', 'joe', 'biden', 'became', 'u.s.', 'president', 'formally', 'proclaim', 'holiday', 'coincides', 'another', 'fall', 'day']
 
all_term_list_deduped: ['peoples', 'fall', '2024', 'partnered', 'family', 'shop.org', 'term', 'shopping', 'tree', 'penguin', 'america', 'public', 'restaurant', '2021', 'cyber', 'google', 'animal', 'juneteenth', 'yorker', 'monday', 'formally', 'nation

# Category Clustering

In the current iteration, I ran the below code a handful of times over data through Season 40 (and, when available, 41) with varying numbers of clusters to be created, each time classifying the clusters manually. Each category thus gets at least that many tags. The distribution of the category tags I assigned becomes the dependent variable in a series of linear regressions, one for each broad group, that I run in the next step of the process.

In [23]:
vectorizer = TfidfVectorizer(stop_words=stopwords_list)
tfidf_matrix = vectorizer.fit_transform(for_cats_std['all_term_list2'])

In [24]:
vector_dict = {}
vector_dict = vectorizer.vocabulary_
#print(“vector_dict:\n”, vector_dict)
# This is really the header for the tfidf_matrix

num_clusters = 250 #i have been doing 250 or 300 clusters usually

print ('\nNum_clusters to be used:', num_clusters)

km = KMeans(n_clusters=num_clusters, init='k-means++', n_init=5, max_iter=50) #you can turn n_init and max_iter down, they add some serious runtime
km.fit(tfidf_matrix)

# Add the cluster number to each comment back to the original DF.
clusters = km.labels_.tolist()
for_cats_std['Cluster_num'] = clusters

#inputDF = inputDF.sort_values([‘Cluster_num’])

print('\nFile_inputDF:\n', for_cats_std)


Num_clusters to be used: 250

File_inputDF:
                    category  \
34124             POTPOURRI   
36949               SCIENCE   
25950            LITERATURE   
20175               HISTORY   
4621       AMERICAN HISTORY   
...                     ...   
44178       THE LAST "OUT\"   
12177              CUBE IT!   
44510            THE MONTHS   
40721   SYNONYMOUS ANAGRAMS   
6336   AUTOS BY THE NUMBERS   

                                           all_term_list  \
34124  [foot, location, metatarsal, arch, potpourri, ...   
36949  [man, mammal, hold, record, longest, lifespan,...   
25950  [tom, depending, book, jones, sawyer, uncle\, ...   
20175  [australia, convicts, british, settler, island...   
4621   [sam, houston, served, congressman, state, sen...   
...                                                  ...   
44178                   [snout, proboscis, last, "out\"]   
12177  [, cube, it!, 27, cube, it!, 216, cube, it!, 1...   
44510     [march, month, start, day, week

The below code outputs all 250 clusters with the highest-frequency words contained within it for easier analysis. I've omitted that output here, but as an example here is what one cluster looks like when run:

246

[('emmy', 91), ('won', 80), ('show', 63), ('series', 58), ('best', 45), ('drama', 43), ('comedy', 41), ('tv', 39)]


['THE EMMYS', 'ENTERTAINMENT AWARDS', '\\"M" & EMMYS', 'JULIE & JULIA', 'EMMY-WINNING WOMEN', '& THE EMMY GOES TO...', 'NEVER WON AN EMMY', 'EMMY WINNERS FOR BEST ACTRESS IN A DRAMA', 'EMMY HISTORY', 'POWER AWARDS', '1950s EMMYS', 'THE EMMY AWARDS', 'EMMY-WINNING TELEVISION MOVIES: REAL LIFE EDITION', 'CANADIAN BORN', 'TV "B"-RUNS', 'THE MARK TWAIN PRIZE FOR AMERICAN HUMOR', 'EMMY WINNERS', 'EMMY-WINNING GUEST APPEARANCES', '2010 EMMY WINNERS', 'THE GOLDEN AGE OF TELEVISION', 'TERRIFIC TV', 'EMMY TIME', 'DRAMA SERIES WRITING EMMYS', "TELL THEM WHAT THEY'VE WON, JOHNNY!", 'EMMYS FOR WRITING', 'THE 2014 EMMYS', 'THE 1955 EMMYS', 'TV ACTORS', 'INITIAL T.V.', 'EMMY M.D.s', 'AN EMMY-WINNING ROLE', 'TELEVISION DRAMA', 'SUPPORTING TV CHARACTERS', 'THE PATRIOT ACT', 'MY TV DADS', 'THE 2011 EMMYS', '1940S TV', 'I WON AN OSCAR, EMMY, TONY & GRAMMY', 'THE 1987 EMMYS', 'CELEBRITY "M.D."s', "THAT'S SHOW BIZ", 'COMEDY ON TV', '>', '60 YEARS OF THE EMMYS', 'THE EMMYs', 'THE 1995 EMMY AWARDS', 'SHOW BIZ AWARD WINNERS', 'SAY "JACK\\"', 'THE 1971 EMMY AWARDS', "IT'S TV, IT'S HBO", '1990s EMMYS', 'HER EMMY-WINNING ROLE', 'THE "X" GAMES', "TV's SUPPORTING ACTORS", "AND THEN THERE'S BEA ARTHUR", 'MY NAME IS BARBARA', 'TV IN THE YEAR 2000', 'EMMY FACTS', "THE ENTERTAINER'S TWITTER BIO", 'TEEN CHOICE AWARDS 2008', 'EARLY EMMYS', 'OF A MAN NAMED BRADY', '1940s TV', 'PHIL-IBUSTER', 'ENTERTAINMENT FIRSTS', 'THE EMMYS 1967', 'THE 1950 EMMYS', 'EMMY REPEAT WINNERS?', '& THE 21st CENTURY EMMY GOES TO...', 'THE DAYTIME EMMYS', 'THE 1999 EMMY AWARDS', "\\'50s TELEVISION", 'TV "P"EOPLE', '1990 EMMYS', 'A SALUTE TO SALLY FIELD', 'MY EMMY-WINNING ROLE', "ACTRESSES' EMMYS", 'SO SAYETH THE TALK SHOW HOST', 'I WON AN EMMY FOR THAT DRAMA', 'THE EMMY FOR...', 'TV DRAMA SERIES', 'THE WINNERS', 'EMMY-WINNING DRAMATIC ACTORS', 'THE 50th ANNUAL EMMY AWARDS', 'TV TALK SHOW HOSTS', 'TV COMEDY CHARACTER NAMES', 'SUPPORTING ACTOR EMMY WINNERS', 'EMMY-WINNING TV', 'TV GUY"D\\"', 'JESSICA', 'RHYMES WITH TICKLE', 'BEST DRAMA SERIES EMMY AWARDS', 'EMMY: THE WOMEN', 'MARY-LOUISE PARKER', "EMMY'S BEST DRAMA", "THE EMMY WINNER'S SHOW", '& THE EMMY FOR COMEDY ACTRESS GOES TO...', 'THEY WON EMMYS FOR...', 'THE EMMY AWARDS 1995', 'THE 1988 EMMYS', "EMMY'S BEST COMEDY SERIES", "EMMY'S BEST COMEDY", 'DAYTIME TV PERSONALITIES', 'ENTERTAINMENT AWARD WINNERS', 'COMEDIC ACTRESSES', 'EMMY: THE MEN', '21st CENTURY EMMYS', "ENTERTAINERS OF THE '60s", 'THE 1991 EMMYS', 'EMMYS FOR COMEDY']

In [ ]:
for i in range(0,num_clusters):
    print(i)
    
    t = list(for_cats_std[for_cats_std.Cluster_num == i]['all_term_list2'])
    pp = [i.split() for i in t]
    oo = [item for sublist in pp for item in sublist]
    oo = [item for item in oo if item.lower() not in stopwords_list]
    counter = collections.Counter(oo)
    print(counter.most_common(8))

    t = list(for_cats_std[for_cats_std.Cluster_num == i]['category'])
    print(t)

I created this set of labels myself for the purposes of studying for Jeopardy, and have redone this several times. Each cluster is assigned up to two labels in a run. Here is how I define each group, with the optional secondary tags I use as applicable - the secondary tags have not been fit to a model yet, but are available in the data:

**Art**

Questions about painters, works of art, and art museums.
* *Secondary tags:* Architecture
* *Example categories:* SEE 'EM AT THE MUSEUM, FAMOUS PAINTINGS, ARTISTS' CHOICE

**Business**

Questions about businesses, the stock market, business leaders, and corporate history.
* *Secondary tags:* Transportation (primarily including questions about carmakers and models, as well as some questions about airlines)
* *Example categories:* STOCK CARS, MERGERS & ACQUISITIONS, FORTUNE 500 COMPANIES

**Civics**

Questions about (usually American) government, politics, law, and courts. As could be expected, this often overlaps with history questions.
* *Secondary tags:* Politics (questions about current events and elected officials)
* *Example categories:* WHAT CABINET DEPARTMENT?, U.S. CONSTITUTION, HISTORIC SUPREME COURT DECISIONS

**Classical Music**

Questions about symphonies, composers, and musical instruments. This does NOT include questions about opera and ballet, which I elected to put in theater; I am willing to reconsider this and move those to this grouping in a future version.
* *Example categories:* YOU'RE NOT ON THE LISZT, THIS ONE TIME AT ORCHESTRA CAMP, GOING FOR BAROQUE
  
**Fashion**

Questions about clothing, designers, and jewelry.
* *Example categories:* DESIGNER INITIALS, FASHION FROM HEAD TO TOE, OOTD

**Film**

Questions about movies, actors, and related awards.
* *Example categories:* OSCAR-WINNING DIRECTORS, THE AFI'S 100 GREATEST AMERICAN MOVIES, GENE WILDER

**Food**

Questions about food, cooking, and beverage, including alcohol.
* *Example categories:* INTERNATIONAL CUISINE, WOULD YOU LIKE A COCKTAIL?, GIRL DINNER

**Geography**

Questions about places, cities, countries, etc. Overlaps with history, as everything that has ever happened has happened in both a time and a place.
* *Secondary tags:* Africa, Asia, Canada, Colleges, Europe, Flags, United States
* *Example categories:* THE LARGEST IN AREA, ALLITERATION ON THE MAP, TOUGH BODIES OF WATER
  
**History**

Questions about events, world leaders, ancient cultures, etc. Overlaps with geography and civics.
* *Secondary tags:* Africa, Canada, Europe, United States, War
* *Example categories:* THE BRITISH MONARCH WHEN..., SIGNERS OF THE DECLARATION OF INDEPENDENCE, HISTORIC QUOTATIONS

**Literature**

Questions about books and authors. Again, not including theater.
* *Secondary tags:* Childrens, Poetry
* *Example categories:* CLASSIC ENGLISH LIT, PUBLISHED POSTHUMOUSLY, HER FIRST PUBLISHED NOVEL
      
**Miscellaneous**

The everything-else tag, created because this data originate as a prep tool and I do not believe categories like "10-letter words" and "before & after" can be easily studied, nor did it fit neatly in the study methods I was using. This is also the home for easily-identifiable but rare subgroups, as can be seen in the secondary tags. There are also many grab-bag or loosely themed categories that do not fit easy classification, most obviously including potpourri and hodgepodge categories.
* *Secondary tags:* Colors, Holidays, Language (questions involving knowing other languages, such as a Spanish vocabulary or French phrases category), Words (10-LETTER WORDS, answers that share a common word, slang, etc.)
* *Example categories:* PEOPLE, ANNUAL EVENTS, ABBREVIATIONS

**Pop Culture**

Categories in which the questions are about the entertainment world generally but do not easily fit into a specific tag; e.g., "this is obviously pop culture but not distinctly film, tv, or music." Tagging like this still keeps these categories out of the "miscellaneous" pool. Also includes less popular media forms like comics and radio. As should be expected, this overlaps heavily with the other pop culture tags. Slowly but surely beginning to include social media and internet culture.
* *Secondary tags:* Celebrities
* *Example categories:* SCI FI, SHOW BIZ AWARD WINNERS, DOWN THE VIDEO TUBE

**Pop Music**

Questions about the pop charts, 20th cenutry music, and beyond. Jazz mostly went here.
* *Example categories:* ROCK ICONS, BILLBOARD CHART-TOPPERS, NO. 1 ALBUMS OF THE '60s

**Religion**

Questions about world religions, religious texts, and practices.
* *Secondary tags:* Christianity, Mythology
* *Example categories:* BIBLICAL QUOTES, GODS & GODDESSES, RELIGIOUS HISTORY

**Science**

Questions about hard sciences and math.
* *Secondary tags:* Animals, Astronomy, Biology, Elements, Math, Medicine, Nature, Plants, Technology
* *Example categories:* THE BODY HUMAN, MAMMALS, DID YOU PLANET THAT WAY?
  
**Sports**

Questions about athletes, games, and sports.
* *Secondary tags:* Toys and Games
* *Example categories:* WE ARE THE CHAMPIONS, BASEBALL ROOKIES OF THE YEAR, NO LONGER AN OLYMPIC SPORT

**Television**

Questions about tv shows, actors, and related awards.
* *Example categories:* EMMY'S BEST DRAMA, IS THERE A TV DOCTOR IN THE HOUSE?, TV CHARACTERS' MAIDEN NAMES

**Theater**

Questions about things that happen on a stage. As mentioned, this is now the cluster I'm most willing to reconsider/rearrange, as opera and ballet will overlap a fair bit with classical music trivia whereas Broadway is more of a "pop" topic.
* *Secondary tags:* Ballet, Broadway, Opera, Shakespeare
* *Example categories:* SOPRANOS, CHARACTERS IN MUSICALS, AMERICAN BALLET THEATRE'S 75th ANNIVERSARY

In [27]:
for_cats_gp = pd.read_csv('cats250_niter5_50_07302025.csv') # previous cluster labels

#x = pd.merge(for_cats_gp, for_cats_std, right_on=['Cluster_num'], left_on=['cluster'])
#x = x[['category', 'all_term_list_x', 'len', 'Cluster_num']]
#x['all_term_list2'] = [" ".join(x) for x in for_cats_gp.all_term_list]
#x.to_csv('cats250_labeled_07302025a.tsv', sep='\t')

# i didn't create labels for the above clusters, but if you do, here's what the labels doc looks like; run the above code to save an iteration

for_cats_gp.sample(10) #example of manual labeling doc, which gets merged to clusters and then categories 

,cluster,cat1,sub1,cat2,sub2
148,148,history,history.europe,NaN,NaN
32,32,science,NaN,miscellaneous,NaN
224,224,film,NaN,pop culture,NaN
146,146,miscellaneous,NaN,NaN,NaN
238,238,science,science.animals,NaN,NaN
8,8,history,NaN,pop music,NaN
71,71,food,NaN,NaN,NaN
2,2,science,science.animals,NaN,NaN
38,38,television,NaN,NaN,NaN
167,167,food,NaN,NaN,NaN


# Consolidating Model Runs

Combining all my cluster labeling attempts allows me to give each category a % of times it has been tagged with a particular group. I'm sure there's a more pythonic way to do this. Do not @ me about it.

In [35]:
pd.read_csv('labels/cats_labeled_250_12292024.tsv', sep='\t').sample(10) #example of label file contents

,Unnamed: 0,cluster,cat1,sub1,cat2,sub2,category,all_term_list,all_term_list_deduped,all_term_list2,len,Cluster_num
6856,6856,35,miscellaneous,miscellaneous.words,NaN,NaN,WRONG TIME,"['tardy', 'latin', 'slow', 'give', 'adjective'...","['refer', 'immature', 'unripe', 'may', 'humor'...",refer immature unripe may humor latin school l...,338,35
6598,6598,33,pop culture,NaN,NaN,NaN,CELEBRITY PALS,"['david', 'beckham', 'posh', 'spice', 'said', ...","['frank', 'club', 'jon', 'went', 'plane', 'two...",frank club jon went plane two federer vegas 20...,489,33
3634,3634,16,history,history.united states,civics,NaN,THE NEW YORK TIMES NEWS OF 2006,"['turin', 'gold', 'hill', 'host', 'city', 'tim...","['tabloids', 'u.s.', 'g.o.p', 'reported', 'sit...",tabloids u.s. g.o.p reported sitting winter ka...,407,16
18548,18548,90,science,science.medicine,NaN,NaN,SCIENTIFIC SOUTH AMERICAN,"['malaria', 'manuel', 'patarroyo', 'developed'...","['alzheimer', 'inventing', 'still', 'promoter'...",alzheimer inventing still promoter american he...,451,90
32135,32135,138,television,NaN,NaN,NaN,TV ALIENS,"['earth', 'passenger', 'battlestar', 'galactic...","['planet', 'galactica', 'hero', 'ralph', 'tv',...",planet galactica hero ralph tv superman greate...,281,138
41874,41874,182,geography,NaN,NaN,NaN,"\""V"" HAVE MAPS","['valley', 'forge', 'prussian', 'baron', 'frie...","['gediminas', 'steuben', 'forge', 'winter', 'm...",gediminas steuben forge winter mang train 19th...,340,182
2449,2449,12,film,NaN,NaN,NaN,BARBRA STREISAND MOVIES,"['meet', 'fockers', '2004', 'barbra', 'streisa...","['movie', 'decides', 'ziegfeld', 'funny', 'mee...",movie decides ziegfeld funny meet fockers slum...,265,12
16506,16506,75,miscellaneous,miscellaneous.words,NaN,NaN,ATTACK OF THE THESAURUS,"['fox', 'outsmart', 'outwit', 'someone', 'out-...","['animal', 'out-', 'prevarication', 'ha', 'the...",animal out- prevarication ha thesaurus titan g...,151,75
26241,26241,116,miscellaneous,NaN,NaN,NaN,CALIFORNIA EXPORTS,"['wines', '90', '%', 'exported', 'u.s.', 'come...","['club', 'u.s.', 'mondavi', '15', 'golf', 'hea...",club u.s. mondavi 15 golf hearing house cochle...,227,116
20109,20109,96,miscellaneous,NaN,NaN,NaN,THE OCCULT,"['white', 'good', 'magic', 'often', 'referred'...","['female', 'cat', 'ability', 'gazed', 'book', ...",female cat ability gazed book occult practice ...,2648,96


In [9]:
# pulling in all instances of my labeling from the current regime of labels

labels_1 = pd.read_csv('labels/cats_labeled_250_12292024.tsv', sep='\t')
labels_2 = pd.read_csv('labels/cats250_labeled_12302024a.tsv', sep='\t')
labels_3 = pd.read_csv('labels/cats250_labeled_12302024b.tsv', sep='\t')
labels_4 = pd.read_csv('labels/cats250_labeled_12312024a.tsv', sep='\t')
labels_5 = pd.read_csv('labels/cats250_labeled_12312024b.tsv', sep='\t')
labels_6 = pd.read_csv('labels/cats250_labeled_01022025a.tsv', sep='\t')
labels_7 = pd.read_csv('labels/cats250_labeled_01042025a.tsv', sep='\t')
labels_8 = pd.read_csv('labels/cats250_labeled_07272025a.tsv', sep='\t')
labels_9 = pd.read_csv('labels/cats250_labeled_07302025a.tsv', sep='\t') #includes season 41

labels_1 = labels_1[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_2 = labels_2[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_3 = labels_3[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_4 = labels_4[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_5 = labels_5[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_6 = labels_6[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_7 = labels_7[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_8 = labels_8[['category', 'cat1', 'sub1', 'cat2', 'sub2']]
labels_9 = labels_9[['category', 'cat1', 'sub1', 'cat2', 'sub2']]

test_merge = pd.merge(labels_1, labels_2, on='category', suffixes=['_a','_b']).fillna('')
test_merge = pd.merge(test_merge, labels_3, on='category', suffixes=['','_c']).fillna('')
test_merge = pd.merge(test_merge, labels_4, on='category', suffixes=['_c','_d']).fillna('')
test_merge = pd.merge(test_merge, labels_5, on='category', suffixes=['_d','_e']).fillna('')
test_merge = pd.merge(test_merge, labels_6, on='category', suffixes=['_e','_f']).fillna('')
test_merge = pd.merge(test_merge, labels_7, on='category', suffixes=['_f','_g']).fillna('')
test_merge = pd.merge(test_merge, labels_8, on='category', suffixes=['_g','_h']).fillna('')
test_merge = pd.merge(test_merge, labels_9, on='category', suffixes=['_h','_i'], how='outer').fillna('')

In [10]:
test_merge.head()

,category,cat1_a,sub1_a,cat2_a,sub2_a,cat1_b,sub1_b,cat2_b,sub2_b,cat1_c,...,cat2_g,sub2_g,cat1_h,sub1_h,cat2_h,sub2_h,cat1,sub1,cat2,sub2
0,75 YEARS OF THE OSCARS,film,,,,film,,,,film,...,,,film,,,,film,,,
1,#,miscellaneous,miscellaneous.words,,,miscellaneous,miscellaneous.words,,,miscellaneous,...,,,literature,literature.childrens,,,literature,,film,
2,#1 HITS,pop music,,,,pop music,,,,pop music,...,,,pop music,,theater,theater.broadway,pop music,,,
3,#1 SONGS,pop music,,,,pop music,,,,pop music,...,,,pop music,,,,pop music,,,
4,$ IN THE NEWS,civics,,,,miscellaneous,,,,civics,...,television,,film,,,,miscellaneous,,,


In [11]:
#creating a list of tags for each category

cats_all = []
subs_all = []

for i,j in test_merge.iterrows():
    y = [j.cat1_a, j.cat2_a, 
         j.cat1_b, j.cat2_b, 
         j.cat1_c, j.cat2_c, 
         j.cat1_d, j.cat2_d, 
         j.cat1_e, j.cat2_e,
         j.cat1_f, j.cat2_f, 
         j.cat1_g, j.cat2_g,
         j.cat1_h, j.cat2_h,
        j.cat1, j.cat2]
    #y = list(set(list(filter(None, y)))) #dedupe and remove nulls
    y = list(filter(None, y)) # just dedupe
    cats_all.append(y)

for i,j in test_merge.iterrows():
    y = [j.sub1_a, j.sub2_a, 
         j.sub1_b, j.sub2_b, 
         j.sub1_c, j.sub2_c, 
         j.sub1_d, j.sub2_d, 
         j.sub1_e, j.sub2_e,
         j.sub1_f, j.sub2_f, 
         j.sub1_g, j.sub2_g,
         j.sub1_h, j.sub2_h,
        j.sub1, j.sub2]
    #y = list(set(list(filter(None, y)))) #dedupe and remove nulls
    y = list(filter(None, y)) # just dedupe
    subs_all.append(y)

test_merge['cats'] = cats_all
test_merge['cats_len'] = [len(i) for i in test_merge.cats] 

test_merge['subs'] = subs_all
test_merge['subs_len'] = [len(i) for i in test_merge.subs] 

In [12]:
#consolidating tags into a 0-1 tag label

test_merge['tag_count'] = [len(i) for i in test_merge['cats']]
test_merge['sub_count'] = [len(i) for i in test_merge['subs']]

                              
def keep_cat(tag, a_list):
    return [i for i in a_list if i == tag]

for tag in tags:
    test_merge[tag] = [round(len(keep_cat(tag, i)) / j,2) for i, j in zip(test_merge['cats'], test_merge['tag_count'])]

In [ ]:
# for the specific case of identifying which sub-population of sports questions is games

gametag = []

for i, j in zip(test_merge['subs'], test_merge['sub_count']):
    try: 
        val = round(len(keep_cat('sports.toys and games', i)) / j,2)
        gametag.append(val)
    except:
        gametag.append(0)

test_merge['games'] = gametag
test_merge['games'] = round(test_merge.games * test_merge.sports,2)
    

In [15]:
test_merge.columns
test_merge = test_merge[['category',  'miscellaneous',
       'art', 'business', 'civics', 'classical music', 'fashion',
       'film', 'food', 'geography', 'history', 'literature',
       'pop culture', 'pop music', 'religion', 'science', 'sports', 'television',
       'theater', 'tag_count']]

test_merge.sample(10).T

,9500,16673,10784,30759,42183,15815,7846,25592,20283,40987
category,CARRYING ON,FOREIGN PHRASEBOOK,CLOSING THE BOOK,NURSERY RHYME THINGS,THE 1970s MUSIC SCENE,FIELD HOCKEY,BLACK'S LAW DICTIONARY SAYS...,LIKE WE DID LAST SUMMER,"HIT THE ""BEACH\""",TAM O'SHATNER
miscellaneous,0.0,1.0,0.22,0.73,0.2,0.0,0.27,0.11,0.18,0.0
art,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
civics,0.0,0.0,0.0,0.0,0.0,0.0,0.55,0.0,0.0,0.0
classical music,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fashion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
film,0.25,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.3
food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
geography,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.36,0.0


In [16]:
# merging the category data back onto this frame

for_cats_std = pd.read_pickle('category_term_lists_s41.pkl')

for_cats_std_with_tags = pd.merge(for_cats_std, test_merge, on='category')

In [17]:
for_cats_std_with_tags.sample(10)

,category,all_term_list,all_term_list_deduped,all_term_list2,len,miscellaneous,art,business,civics,classical music,...,ushistory,literature,pop culture,pop music,religion,science,sports,television,theater,tag_count
41776,"IRON, MAN","[golf, iron, used, sport, may, either, cast, f...","[man, used, element, iron,, protein, necessary...","man used element iron, protein necessary steel...",268,0.11,0.00,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.89,0.0,0.0,0.00,9
39473,HOLLYWOOD FRUITS,"[grapefruit, cagney, going, shove, omelet, may...","[dream, strawberry, disappeared, humphrey, mut...",dream strawberry disappeared humphrey mutiny\ ...,281,0.18,0.00,0.00,0.0,0.0,...,0.0,0.36,0.00,0.00,0.00,0.00,0.0,0.0,0.00,11
26786,RADIO DISNEY TOP 30,"[hilary, duff, sept., 2004, top, 30, 11, 13, 1...","[official, lavigne, lindsay, sister, warning, ...",official lavigne lindsay sister warning movie ...,344,0.00,0.00,0.00,0.0,0.0,...,0.0,0.00,0.10,0.60,0.00,0.10,0.1,0.1,0.00,10
54840,THE WORLD'S PEOPLE,"[catholic, 1st, recorded, use, word, applying,...","[1st, billion, use, world's, word, 1.1, around...",1st billion use world's word 1.1 around 100 st...,102,0.09,0.00,0.09,0.0,0.0,...,0.0,0.00,0.00,0.00,0.64,0.00,0.0,0.0,0.00,11
93,STATE CAPITALS,"[connecticut, 1875, dual, capital, new, haven,...","[land, 12, wind, 24, 1971, nbc, 30, library, d...",land 12 wind 24 1971 nbc 30 library dakota n s...,14687,0.70,0.00,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,10
12262,INTERESTING PEOPLE,"[home, run, sadaharu, oh, yomiuri, giants, hit...","[organization, fossil, france, dantas, brazili...",organization fossil france dantas brazilian oh...,436,0.33,0.00,0.00,0.0,0.0,...,0.0,0.00,0.08,0.08,0.00,0.08,0.0,0.0,0.00,12
49453,MOVIE STARS SPEAK,"[elijah, wood, admitted, read, lord, rings, ag...","[movie, set, play, read, year, believing, ask,...",movie set play read year believing ask lord mu...,204,0.00,0.00,0.00,0.0,0.0,...,0.0,0.10,0.00,0.00,0.00,0.00,0.0,0.1,0.00,10
25913,DESCRIBING THE NO. 1 SONG,"[vogue, 1990, madonna, turn, magazine, dance, ...","[m, boy, country, enough, baby, grateful, cent...",m boy country enough baby grateful centerfold ...,349,0.25,0.00,0.00,0.0,0.0,...,0.0,0.00,0.25,0.42,0.00,0.00,0.0,0.0,0.08,12
46627,SPOUSE IN COMMON TO...,"[steven, spielberg, amy, irving, kate, capshaw...","[kathryn, michelle, irving, capshaw, spielberg...",kathryn michelle irving capshaw spielberg smit...,232,0.08,0.00,0.00,0.0,0.0,...,0.0,0.00,0.08,0.31,0.00,0.00,0.0,0.0,0.00,13
46166,RELIGION IN ART,"[noah, ark, famous, peaceable, kingdom, series...","[famous, gustav, church, 19th, art, artist, lo...",famous gustav church 19th art artist location ...,236,0.00,0.78,0.00,0.0,0.0,...,0.0,0.00,0.00,0.00,0.11,0.00,0.0,0.0,0.00,9


# Linear Regression on Cluster Tags

For each broad genre of topic, I fit a linear regression. Each word's presence or absence is a feature, and the 0-1 value for that tag generated above is the dependent variable. To validate, I output the high-coefficient ("important") features for the model, as well as the topics with the strongest predicted value for that model. There's some noise among the list of highest-value features I can't quite iron out, but that doesn't seem to create problems with the outputs.

I have not yet created separate models for the secondary tags ('opera' within theater, 'chemistry' within science, etc.) that would make the labels more granular, although those tags are included in the "labels" files and do not need any manual labeling from someone else to be created.

In [35]:
just_cats = for_cats_std_with_tags[['category']]

for tag in tags:
    df_spare = for_cats_std_with_tags[[tag, 'all_term_list2']]
    df_spare
    
    target_array_full = np.asarray(df_spare[tag])
    corpus = [str(item).lower() for item in df_spare['all_term_list2']]
    
    tfid_max = int(len(df_spare['all_term_list2'])*.9)
    tfid_vectorizer_all = TfidfVectorizer(ngram_range=(1,1), min_df=32, max_df=tfid_max, stop_words=stopwords_list)
    full_corpus_vectorizer = tfid_vectorizer_all.fit(corpus)
    feature_matrix = tfid_vectorizer_all.transform(corpus)
    
    #print(feature_matrix.shape)
    #print(target_array_full.shape)

    lreg = LinearRegression()
    lreg.fit(feature_matrix, target_array_full)
    coefficients = list(lreg.coef_)
    
    feature_names = tfid_vectorizer_all.get_feature_names_out()
    df = pd.DataFrame(list(zip(feature_names, coefficients)), 
                   columns =['Feature', 'Coefficient']) 
    
    df_nonzero = df[df['Coefficient'] != 0].sort_values('Coefficient', ascending=False) #only shows non-zero coefficients

    # print the name of the category, highest-coeffieicent features for each category, and the categories most confidently labeled by the model
    
    print(tag)
    print(list(df_nonzero['Feature'][:25]))

    fieldname = tag + '_predict'
    just_cats[fieldname] = lreg.predict(feature_matrix)
    
    print(list(just_cats.sort_values(fieldname, ascending=False)['category'].head(25)))

art
['vinci', 'painting', 'art', 'magritte', 'whistler', 'gogh', 'painted', 'monet', 'edvard', 'manet', 'matisse', 'wyeth', 'rembrandt', 'painter', 'cather', 'moma', 'rococo', 'guernica', 'dali', 'picasso', 'rodin', 'portrait', 'vermeer', 'heston', 'cubism']
['FAMOUS PAINTINGS', "SEE 'EM AT THE MUSEUM", "ARTISTS' CHOICE", 'PAINTERS', 'FAMOUS PORTRAITS', 'PAINTINGS', 'THE MASTERS', 'FILL IN THE BLANK CANVAS', 'IMPRESSIONISTS', 'PAINTERS & PAINTINGS', 'THE ART WORLD', 'ART "C\\"', 'ART MOVEMENTS', 'WHERE FOR ART?', 'ARTISTIC QUOTES', 'ARTISTS & THEIR WORKS', 'EUROPEAN PAINTERS', "I PROBABLY COULDN'T PAINT THAT", 'YOU GOTTA HAVE ART', '\\"P"AINTERS & "P"AINTING', 'AROUND THE LOUVRE', '\\"R"T', 'ALL ABOUT ART', 'ARTISTS IN EUROPE', 'MASTERS']
business
['company', 'stock', 'toyota', 'car', 'pontiac', 'minted', 'airline', 'honda', 'investment', 'buick', 'ferrari', 'paltrow', 'suv', 'mustang', 'volkswagen', 'retailer', 'nasdaq', 'airlines', 'volvo', 'edsel', 'peso', 'business', 'telecom', 'kl

In [36]:
# all the predicted values in one frame and maximum predicted value

just_cats['max_predict'] = just_cats.iloc[:,1:19].idxmax(axis=1).str.replace('_predict','')
just_cats['second_max_predict'] = just_cats.iloc[:, 1:19].apply(lambda row: row.nlargest(2).idxmin(), axis=1).str.replace('_predict','')

#just_cats.to_csv('just_cats_linear_reg_08022025.csv') #to save the output
just_cats.head()

,category,art_predict,business_predict,civics_predict,classical music_predict,fashion_predict,film_predict,food_predict,geography_predict,history_predict,...,miscellaneous_predict,pop culture_predict,pop music_predict,religion_predict,science_predict,sports_predict,television_predict,theater_predict,max_predict,second_max_predict
0,POTPOURRI,-0.029759,0.096773,-0.025515,0.075611,0.080765,-0.109308,0.092537,-0.034043,-0.025516,...,0.897951,0.044325,-0.090573,0.021820,0.178650,-0.017369,-0.035231,-0.034387,miscellaneous,science
1,SCIENCE,-0.046719,-0.052214,-0.037531,-0.013426,-0.040556,-0.102427,-0.075252,-0.128446,-0.237420,...,0.505226,-0.042338,-0.099393,-0.078480,1.682990,-0.030361,-0.063732,-0.028780,science,miscellaneous
2,LITERATURE,-0.024625,-0.021783,-0.042847,0.008082,-0.061100,-0.008865,-0.026848,-0.151515,-0.151557,...,0.623776,-0.022784,-0.098462,-0.049602,-0.170063,0.061940,-0.054588,0.073249,literature,miscellaneous
3,HISTORY,-0.060064,-0.016574,-0.017587,-0.016052,0.003580,0.000356,-0.024449,0.093253,0.837478,...,0.844460,-0.081791,-0.087518,-0.058953,-0.135501,-0.092426,-0.012939,-0.008531,miscellaneous,history
4,AMERICAN HISTORY,-0.056226,0.014678,0.039657,-0.004201,-0.030702,0.005750,-0.029202,-0.048329,0.631631,...,0.903155,-0.018148,0.013504,-0.009155,-0.091708,-0.086370,0.006012,-0.047031,miscellaneous,history


I also made the following manual edits on the output file:

1. Remove all secondary tags of "miscellaneous"
2. Remove all secondary tags where the gap between the highest and second-highest predicted value is greater than 0.5
3. Replaced "miscellaneous" primary tags  with the secondary predicted tag on categories with especially high secondary predicted values
4. Reviewed remaining "miscellaneous" categories in the top 500 and edited where needed
5. Removed all remaining secondary tags <0.19 predicted score
6. Removed all remaining secondary tags <0.3 predicted score and gap greater than 0.1
7. Miscellaneous additional ad hoc edits

If you do this yourself, don't worry too much about the accuracy of categories below the first couple thousand; nearly every category name is only used once, so a stray "Puccini" in an otherwise miscellaneous category could get it flagged as opera. The top 10,000 categories (of 55,000) comprise approximately 60% of the total clue corpus. As well, short of a systematic review nothing I have done to modify the labels has significantly impacted the takeaways from the below applications (get rate order doesn't change significantly, etc.)

# Applications of Labeled Category Data

### Category Get Rates

Once again: I am aware my code sucks.

This is code that imports "processed games 9 4" - a dataset including details about game flow and correct answers - and marks clues for whether _at least one contestant_ answered the question correctly.

In [23]:
category_tags = pd.read_csv('cats_with_tags_final_2025.csv')
game_flow_data = pd.read_csv('processed_games_9_4.csv')

j = pd.read_pickle('s41_cleaned.pkl')
questions_with_cats = pd.merge(j, category_tags, on='category')
questions_with_cats['year'] = [int(i[:4]) for i in questions_with_cats.air_date]
questions_with_cats['daily_double'] = [i > 0 for i in questions_with_cats.daily_double_value]

In [24]:
questions_trunc = questions_with_cats[['air_date', 'round', 'category', 'answer', 'question', 'clue_value', 'max_predict', 'second_max_predict']]
gameflow_trunc = game_flow_data[['airdate', 'clue_text', 'clue_values', 'clue_answers', 'player1_right', 'player2_right',
       'player3_right', 'player1_wrong', 'player2_wrong', 'player3_wrong', 'dd_flag']]

getrate = questions_trunc.merge(gameflow_trunc, left_on=['air_date', 'question'], right_on=['airdate', 'clue_answers'], how='inner')

getrate['correct'] = getrate['player1_right'] + getrate['player2_right'] + getrate['player3_right']
getrate['wrong'] = getrate['player1_wrong'] + getrate['player2_wrong'] + getrate['player3_wrong']

In [25]:
getrate['two_tags'] = getrate.second_max_predict.notnull()
getrate['correct_weight'] = [0.5 if i == True else 1.0 for i in getrate.two_tags] 
getrate['correct_value'] = getrate.correct * getrate.correct_weight
getrate['claimed_dollars'] = getrate.correct_value * getrate.clue_value 
getrate['claimable_dollars'] = getrate.correct_weight * getrate.clue_value 
getrate.sample(5)

,air_date,round,category,answer,question,clue_value,max_predict,second_max_predict,airdate,clue_text,...,player2_wrong,player3_wrong,dd_flag,correct,wrong,two_tags,correct_weight,correct_value,claimed_dollars,claimable_dollars
101789,2012-01-10,2,COMICS STRIP,Ken Jeong bared all as crime lord Mr. Chow in ...,The Hangover,2000,pop culture,NaN,2012-01-10,Ken Jeong bared all as crime lord Mr. Chow in ...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,2000.0,2000.0
173684,2018-12-24,1,NOT FOR PROFIT,The NRDC fights for clean energy & against pow...,coal,200,science,NaN,2018-12-24,The NRDC fights for clean energy & against pow...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,200.0,200.0
148597,2016-07-22,1,THE BUCKET LIST,Bucket seats in cars were popularized by this ...,Chevy,800,film,NaN,2016-07-22,Bucket seats in cars were popularized by this ...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,800.0,800.0
222462,2023-07-05,1,WORLD OF FOOD,This name for a pizza turnover comes from the ...,calzone,200,food,NaN,2023-07-05,This name for a pizza turnover comes from the ...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,200.0,200.0
3768,2002-04-24,1,"LARRY, MOE OR CURLY",Leisure-suited lounge lizard who's starred in ...,Larry,800,television,NaN,2002-04-24,Leisure-suited lounge lizard who's starred in ...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,800.0,800.0


In [27]:
getrate[['air_date','round','clue_value','category','answer','question','max_predict', 'second_max_predict']].sample(5)

,air_date,round,clue_value,category,answer,question,max_predict,second_max_predict
134276,2015-04-15,2,2000,COMPOUND WORDS,A ship's cockpit is this enclosed area on the ...,wheelhouse,miscellaneous,NaN
33950,2005-09-12,1,200,"FRED THOMPSON, ACTING SENATOR",Fred Dalton Thompson was still in the Senate w...,Law & Order,civics,film
111310,2012-12-31,1,200,PEOPLE ARE READING...,"\""An American Son"", by this senator whose pare...",Marco Rubio,literature,pop music
104110,2012-04-04,1,1000,KISSING,Ryabovich gets a kiss by mistake from a myster...,Chekhov,literature,NaN
45964,2006-10-12,1,600,A LITERARY TOUR,"Have a homey lunch at this author's Salinas, C...",John Steinbeck,literature,NaN


This data only covers games since 2002.

In [28]:
min(getrate['air_date'])

'2002-01-01'

In [29]:
max(getrate['air_date'])

'2025-07-25'

In [33]:
get_rates_cat = pd.DataFrame(columns=['category', 'round', 'value', 'getrate', 'correct', 'clues'])

for roundd in [1,2]:
    if roundd == 1:
        for clue_value in [200,400,600,800,1000]:
            subframe = getrate[(getrate['round'] == 1) & (getrate.clue_value == clue_value)]
            numer = subframe.correct_value.sum()
            denom = subframe.correct_weight.sum()

            correct_pct = numer / denom

            row = {'category': 'overall', 'round': roundd, 'value': clue_value, 'getrate': correct_pct, 'correct': numer, 'clues': denom}
            get_rates_cat.loc[len(get_rates_cat)] = row
            
    if roundd == 2:
        for clue_value in [400,800,1200,1600,2000]:
            subframe = getrate[(getrate['round'] == 2) & (getrate.clue_value == clue_value)]
            numer = subframe.correct_value.sum()
            denom = subframe.correct_weight.sum()

            correct_pct = numer / denom

            row = {'category': 'overall', 'round': roundd, 'value': clue_value, 'getrate': correct_pct, 'correct': numer, 'clues': denom}
            get_rates_cat.loc[len(get_rates_cat)] = row
                
    #print(str(tag) + ': ' + str(tag_frame.clue_value.mean()))

get_rates_cat

,category,round,value,getrate,correct,clues
0,overall,1,200,0.959177,21322.5,22230.0
1,overall,1,400,0.918826,20278.5,22070.0
2,overall,1,600,0.881075,19407.0,22026.5
3,overall,1,800,0.840175,18462.0,21974.0
4,overall,1,1000,0.750973,16299.5,21704.5
5,overall,2,400,0.936496,20395.0,21778.0
6,overall,2,800,0.877490,19070.5,21733.0
7,overall,2,1200,0.813987,17650.5,21684.0
8,overall,2,1600,0.745358,16118.0,21624.5
9,overall,2,2000,0.645195,13672.0,21190.5


Generating the by-dollar value and label crosstab:

In [12]:
get_rates_cat = pd.DataFrame(columns=['category', 'round', 'value', 'getrate', 'correct', 'clues'])

for tag in tags:
    tag_frame = getrate[(getrate.max_predict == tag) | (getrate.second_max_predict == tag)]
    #tag_frame = tag_frame[tag_frame.air_date > '2020-01-01']

    for roundd in [1,2]:
        if roundd == 1:
            for clue_value in [200,400,600,800,1000]:
                subframe = tag_frame[(tag_frame['round'] == 1) & (tag_frame.clue_value == clue_value)]
                numer = subframe.correct_value.sum()
                denom = subframe.correct_weight.sum()
    
                correct_pct = numer / denom

                row = {'category': tag, 'round': roundd, 'value': clue_value, 'getrate': correct_pct, 'correct': numer, 'clues': denom}
                get_rates_cat.loc[len(get_rates_cat)] = row
                
        if roundd == 2:
            for clue_value in [400,800,1200,1600,2000]:
                subframe = tag_frame[(tag_frame['round'] == 2) & (tag_frame.clue_value == clue_value)]
                numer = subframe.correct_value.sum()
                denom = subframe.correct_weight.sum()
    
                correct_pct = numer / denom
    
                row = {'category': tag, 'round': roundd, 'value': clue_value, 'getrate': correct_pct, 'correct': numer, 'clues': denom}
                get_rates_cat.loc[len(get_rates_cat)] = row
                
    #print(str(tag) + ': ' + str(tag_frame.clue_value.mean()))

get_rates_cat

,category,round,value,getrate,correct,clues
0,art,1,200,0.964623,204.5,212.0
1,art,1,400,0.919048,193.0,210.0
2,art,1,600,0.852500,170.5,200.0
3,art,1,800,0.781863,159.5,204.0
4,art,1,1000,0.650485,134.0,206.0
...,...,...,...,...,...,...
175,theater,2,400,0.928773,658.5,709.0
176,theater,2,800,0.858965,606.0,705.5
177,theater,2,1200,0.797450,563.0,706.0
178,theater,2,1600,0.722384,497.0,688.0


In [16]:
getrate.head()

,air_date,round,category,answer,question,clue_value,max_predict,second_max_predict,airdate,clue_text,...,player2_wrong,player3_wrong,dd_flag,correct,wrong,two_tags,correct_weight,correct_value,claimed_dollars,claimable_dollars
0,2002-01-01,1,AMERICAN MUSEUMS,The history of evangelism is on display at a W...,Billy Graham,200,geography,NaN,2002-01-01,The history of evangelism is on display at a W...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,200.0,200.0
1,2002-01-01,1,AMERICAN MUSEUMS,"A visit to this soft drink's Waco, Texas museu...",Dr Pepper,400,geography,NaN,2002-01-01,"A visit to this soft drink's Waco, Texas museu...",...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,400.0,400.0
2,2002-01-01,1,AMERICAN MUSEUMS,"The last Major Leaguer to hit .400, his Museum...",Ted Williams,600,geography,NaN,2002-01-01,"The last Major Leaguer to hit .400, his Museum...",...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,600.0,600.0
3,2002-01-01,1,AMERICAN MUSEUMS,Exhibits at Seattle's Museum of Flight are hou...,Boeing,800,geography,NaN,2002-01-01,Exhibits at Seattle's Museum of Flight are hou...,...,0.0,0.0,0,1.0,0.0,False,1.0,1.0,800.0,800.0
4,2002-01-01,1,AMERICAN MUSEUMS,"Built from 2 slave cabins, the Uncle Remus Mus...",Joel Chandler Harris,1000,geography,NaN,2002-01-01,"Built from 2 slave cabins, the Uncle Remus Mus...",...,0.0,0.0,0,0.0,0.0,False,1.0,0.0,0.0,1000.0


In [18]:
query = """
select max_predict as 'tag', count(*) as clue_count, avg(clue_value) as value
from getrate
where air_date >= '2011-01-01'
group by max_predict
UNION ALL
select second_max_predict as 'tag', count(*) as clue_count, avg(clue_value) as value
from getrate
where air_date >= '2011-01-01'
group by second_max_predict
"""

sqldf(query).to_csv('freq_value.csv')


In [653]:
query = """
select category, round, 'tot.' as value, sum(correct) / sum(clues) as 'getrate', sum(correct) as correct, sum(clues) as clues
from get_rates_cat
group by category, round
UNION ALL
select category, 'tot.' as round, 'tot.' as value, sum(correct) / sum(clues) as 'getrate', sum(correct) as correct, sum(clues) as clues
from get_rates_cat
group by category
"""

round_and_game = sqldf(query)

overall_pivot = round_and_game.pivot(index='category', columns=['round', 'value'])['getrate']

overall_pivot.columns = ['r1_overall', 'r2_overall', 'game_overall']

overall_pivot

,r1_overall,r2_overall,game_overall
category,,,
art,0.834787,0.762655,0.784469
business,0.873002,0.805150,0.849225
civics,0.865390,0.812227,0.840006
classical music,0.867762,0.763201,0.798930
fashion,0.861571,0.770285,0.835351
film,0.897778,0.832265,0.860796
food,0.879623,0.835254,0.867306
geography,0.861584,0.794993,0.827925
history,0.856423,0.788117,0.819853


In [692]:
rate_pivot = get_rates_cat.pivot(index='category', columns=['round', 'value'])['getrate']

rate_pivot.columns = ['1-200', '1-400', '1-600', '1-800', '1-1000', '2-400', '2-800', '2-1200', '2-1600', '2-2000']

overall_pivot.join(rate_pivot, how='left').sort_values('game_overall', ascending=False).to_csv('getrates_12282025.csv')

In [656]:
# alternative view not used in article: pct of all claimed dollars

for tag in tags:
    tag_frame = getrate[(getrate.max_predict == tag) | (getrate.second_max_predict == tag)]
    #tag_frame = tag_frame[tag_frame.air_date > '2020-01-01

    subframe = tag_frame
    numer = subframe.claimed_dollars.sum()
    denom = subframe.claimable_dollars.sum()
    
    correct_pct = numer / denom

    print(str(tag) +  '|' + str(correct_pct)[:5] + '|' + str(numer) + '|' + str(denom))
    #print(str(tag) + ': ' + str(tag_frame.clue_value.mean()))


art|0.717|2475000.0|3451200.0
business|0.800|4037800.0|5043500.0
civics|0.789|2502400.0|3168100.0
classical music|0.729|1347000.0|1846300.0
fashion|0.778|982500.0|1261400.0
film|0.810|9351000.0|11530600.0
food|0.823|5297300.0|6430600.0
geography|0.772|20574700.0|26644400.0
history|0.760|23867800.0|31365500.0
literature|0.733|12969600.0|17681800.0
miscellaneous|0.807|37114200.0|45950600.0
pop culture|0.813|2391100.0|2938500.0
pop music|0.792|5980600.0|7550400.0
religion|0.777|3619600.0|4657000.0
science|0.787|19056800.0|24209700.0
sports|0.818|5579900.0|6821300.0
television|0.826|5824400.0|7051200.0
theater|0.742|3809900.0|5129300.0


### Average Clue Values

This looks at the average value of clues given the category labels I created.

In [657]:
qwc_sql.date.dt.year

0         1984
1         1984
2         1984
3         1984
4         1984
          ... 
529418    2025
529419    2025
529420    2025
529421    2025
529422    2025
Name: date, Length: 529423, dtype: int32

In [662]:
questions_with_cats['date'] = pd.to_datetime(questions_with_cats['air_date'], format='%Y-%m-%d')
qwc_sql = questions_with_cats[['category', 'round', 'clue_value', 'max_predict', 'second_max_predict', 'question_modified', 'date']]

qwc_sql = qwc_sql[qwc_sql['round'] != 3]
qwc_sql['clue_value'] = [i * 2 if j <= pd.Timestamp('2001-11-26') else i for i, j in zip(qwc_sql.clue_value, qwc_sql.date)]
qwc_sql['year'] = qwc_sql.date.dt.year

qwc_sql

,category,round,clue_value,max_predict,second_max_predict,question_modified,date,year
0,LAKES & RIVERS,1,200,geography,NaN,jordan,1984-09-10,1984
1,LAKES & RIVERS,1,400,geography,NaN,loch,1984-09-10,1984
2,LAKES & RIVERS,1,800,geography,NaN,missouri,1984-09-10,1984
3,LAKES & RIVERS,1,1000,geography,NaN,caspian sea,1984-09-10,1984
4,INVENTIONS,1,200,science,NaN,radio,1984-09-10,1984
...,...,...,...,...,...,...,...,...
529417,16-LETTER WORDS,2,400,miscellaneous,NaN,counterclockwise,2025-07-25,2025
529418,16-LETTER WORDS,2,800,miscellaneous,NaN,quadricentennial,2025-07-25,2025
529419,16-LETTER WORDS,2,1200,miscellaneous,NaN,disqualification,2025-07-25,2025
529420,16-LETTER WORDS,2,1600,miscellaneous,NaN,environmentalist,2025-07-25,2025


In [664]:
clue_decade_value = pd.DataFrame(columns=['decade', 'category', 'avg value'])

for tag in tags:
    print(tag)
    sqldf_tag = qwc_sql[(qwc_sql.max_predict == tag) | (qwc_sql.second_max_predict == tag)]
    
    query = f"""
        select 
        FLOOR(year / 10) * 10 as 'decade', avg(clue_value) as avg_value
        from sqldf_tag
        group by FLOOR(year / 10) * 10
        union all
        select
        'overall' as 'decade', avg(clue_value) as avg_value
        from sqldf_tag
        """
    yy = sqldf(query)

    for i in range(0, len(yy)):
        row = {'decade': yy['decade'][i], 'category': tag, 'avg value': yy['avg_value'][i]}
        clue_decade_value.loc[len(clue_decade_value)] = row

clue_decade_value

art
business
civics
classical music
fashion
film
food
geography
history
literature
miscellaneous
pop culture
pop music
religion
science
sports
television
theater


,decade,category,avg value
0,1980,art,978.310502
1,1990,art,1017.973231
2,2000,art,1000.781250
3,2010,art,1007.017544
4,2020,art,1009.708738
...,...,...,...
103,1990,theater,1064.240857
104,2000,theater,1012.591152
105,2010,theater,998.360656
106,2020,theater,986.387996


In [665]:
clue_decade_value.to_csv('clue_value_decade_12282025b.csv')

### Board Category Correlations

This is the basis for my claim that there is usually one pop category per round and one literature category per game - I looked at the assigned tags within each jeopardy and double jeopardy board and ran correlations, plus i just summed the number of "sports," "television," "film," "pop music," and "pop culture" tags within each round. I regret not splitting out Broadway from theater but based on my 2021-22 analysis where I I don't think that'd make a huge difference.

In [671]:
questions_with_cats = pd.merge(j, category_tags, on='category')

category_corr = questions_with_cats[questions_with_cats['round'] != 3][['air_date', 'category', 'round', 'max_predict', 'second_max_predict']]

category_corr['encoding_1'] = category_corr['max_predict'] + '_' + category_corr['round'].astype(str)
category_corr['encoding_2'] = category_corr['second_max_predict'] + '_' + category_corr['round'].astype(str)

category_corr = category_corr[['air_date', 'encoding_1', 'encoding_2']].drop_duplicates()

merge1 = category_corr[['air_date', 'encoding_1']]
merge2 = category_corr[['air_date', 'encoding_2']]
merge1.columns = ['air_date', 'encoding']
merge2.columns = ['air_date', 'encoding']

for_encoding = pd.concat([merge1, merge2])
for_encoding = for_encoding[~for_encoding.encoding.isnull()]

encoded = pd.get_dummies(for_encoding, columns = ['encoding'])

encoded.head()

,air_date,encoding_art_1,encoding_art_2,encoding_business_1,encoding_business_2,encoding_civics_1,encoding_civics_2,encoding_classical music_1,encoding_classical music_2,encoding_fashion_1,...,encoding_religion_1,encoding_religion_2,encoding_science_1,encoding_science_2,encoding_sports_1,encoding_sports_2,encoding_television_1,encoding_television_2,encoding_theater_1,encoding_theater_2
0,1984-09-10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1984-09-10,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
13,1984-09-10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17,1984-09-10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22,1984-09-10,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [672]:
encoded.columns

Index(['air_date', 'encoding_art_1', 'encoding_art_2', 'encoding_business_1',
       'encoding_business_2', 'encoding_civics_1', 'encoding_civics_2',
       'encoding_classical music_1', 'encoding_classical music_2',
       'encoding_fashion_1', 'encoding_fashion_2', 'encoding_film_1',
       'encoding_film_2', 'encoding_food_1', 'encoding_food_2',
       'encoding_geography_1', 'encoding_geography_2', 'encoding_history_1',
       'encoding_history_2', 'encoding_literature_1', 'encoding_literature_2',
       'encoding_miscellaneous_1', 'encoding_miscellaneous_2',
       'encoding_pop culture_1', 'encoding_pop culture_2',
       'encoding_pop music_1', 'encoding_pop music_2', 'encoding_religion_1',
       'encoding_religion_2', 'encoding_science_1', 'encoding_science_2',
       'encoding_sports_1', 'encoding_sports_2', 'encoding_television_1',
       'encoding_television_2', 'encoding_theater_1', 'encoding_theater_2'],
      dtype='object')

In [673]:
encoded.columns = ['air_date', 'art_1', 'art_2', 'business_1',
       'business_2', 'civics_1', 'civics_2',
       'classical music_1', 'classical music_2',
       'fashion_1', 'fashion_2', 'film_1',
       'film_2', 'food_1', 'food_2',
       'geography_1', 'geography_2', 'history_1',
       'history_2', 'literature_1', 'literature_2',
       'miscellaneous_1', 'miscellaneous_2',
       'pop culture_1', 'pop culture_2',
       'pop music_1', 'pop music_2', 'religion_1',
       'religion_2', 'science_1', 'science_2',
       'sports_1', 'sports_2', 'television_1',
       'television_2', 'theater_1', 'theater_2']

less_date = ['art_1', 'art_2', 'business_1',
       'business_2', 'civics_1', 'civics_2',
       'classical music_1', 'classical music_2',
       'fashion_1', 'fashion_2', 'film_1',
       'film_2', 'food_1', 'food_2',
       'geography_1', 'geography_2', 'history_1',
       'history_2', 'literature_1', 'literature_2',
       'miscellaneous_1', 'miscellaneous_2',
       'pop culture_1', 'pop culture_2',
       'pop music_1', 'pop music_2', 'religion_1',
       'religion_2', 'science_1', 'science_2',
       'sports_1', 'sports_2', 'television_1',
       'television_2', 'theater_1', 'theater_2']

In [674]:
encoded = encoded.groupby(by='air_date').sum()

In [675]:
encoded['j_tags'] = encoded[['art_1', 'business_1', 'civics_1', 'classical music_1', 'fashion_1',  'film_1',\
                            'food_1', 'geography_1', 'history_1', 'literature_1', 'miscellaneous_1', 'pop culture_1', \
                             'pop music_1', 'religion_1','science_1', 'sports_1',  'television_1', 'theater_1']].sum(axis=1)
encoded['dj_tags'] = encoded[['art_2', 'business_2', 'civics_2', 'classical music_2', 'fashion_2',  'film_2',\
                            'food_2', 'geography_2', 'history_2', 'literature_2', 'miscellaneous_2', 'pop culture_2', \
                             'pop music_2', 'religion_2','science_2', 'sports_2',  'television_2', 'theater_2']].sum(axis=1)

encoded['pop_count_1'] =  encoded[['film_1', 'pop culture_1', 'pop music_1', 'sports_1', 'television_1']].sum(axis=1)
encoded['pop_count_2'] =  encoded[['film_2', 'pop culture_2', 'pop music_2', 'sports_2', 'television_2']].sum(axis=1)

encoded['pop1_normalized'] = (encoded.pop_count_1 / encoded.j_tags) * 6 #pop categories per round accounting for double-tagged cats
encoded['pop2_normalized'] = (encoded.pop_count_2 / encoded.dj_tags) * 6


encoded[['pop_count_1', 'pop_count_2', 'pop1_normalized', 'pop2_normalized']].to_csv('pop_count_12262025.csv')

1.25 "pop" categories per j board and 1.1 "pop" categories per dj board since 1-1-2016, probably a scooch higher because broadway is pop but shakespeare is not (and i did not include theater)

In [676]:
encoded.to_csv('encoded_12262025.csv')

holy shit you are such a nerd are you still reading this

In [677]:
encoded = encoded.drop(['pop_count_1', 'pop_count_2', 'pop_count_1', 'pop_count_2', 'pop1_normalized', 'pop2_normalized'], axis=1)

In [678]:
correlation_matrix = encoded.corr()

correlation_matrix.melt(ignore_index=False).reset_index().to_csv('corr_matrix_12262025.csv')

In [679]:
# most strongly negatively correlated category-round combos. notable findings i probably didn't have space for in the article are 
# "there will be probably not be another literature category"
# the way tv/pop music/film all avoid each other within the same round
# the way sports categories also behave that way

melted = correlation_matrix.melt(ignore_index=False).reset_index().drop_duplicates(subset=['value']).sort_values('value')

melted[melted.value != 1].head(25)

,index,variable,value
703,literature_2,literature_1,-0.309532
391,film_2,film_1,-0.195468
443,pop music_2,film_2,-0.188528
412,television_1,film_1,-0.175503
937,pop music_2,pop music_1,-0.168414
547,geography_2,geography_1,-0.167109
1093,science_2,science_1,-0.165080
944,television_1,pop music_1,-0.163793
757,theater_2,literature_2,-0.162828
1172,television_1,sports_1,-0.143851


### New Answerlines

In [53]:
base_date = pd.Timestamp('2025-08-01')

questions_with_cats['date'] = pd.to_datetime(questions_with_cats['air_date'], format='%Y-%m-%d')
#questions_with_cats['time_since'] = (questions_with_cats.date - base_date).dt.days * -1

questions_with_cats

qwc_sql = questions_with_cats[['category', 'round', 'clue_value', 'max_predict', 'second_max_predict', 'question_modified', 'date']]

qwc_sql['year'] = [i.year for i in qwc_sql['date']]

qwc_sql.head()

,category,round,clue_value,max_predict,second_max_predict,question_modified,date,year
0,LAKES & RIVERS,1,100,geography,NaN,jordan,1984-09-10,1984
1,LAKES & RIVERS,1,200,geography,NaN,loch,1984-09-10,1984
2,LAKES & RIVERS,1,400,geography,NaN,missouri,1984-09-10,1984
3,LAKES & RIVERS,1,500,geography,NaN,caspian sea,1984-09-10,1984
4,INVENTIONS,1,100,science,NaN,radio,1984-09-10,1984


In [54]:
qwc_sql[(qwc_sql.max_predict == 'sports') & (qwc_sql.clue_value == 2000)].sample(5)

,category,round,clue_value,max_predict,second_max_predict,question_modified,date,year
415808,CASINO,2,2000,sports,NaN,crap table,2017-02-01,2017
277105,SPORTS STARS,2,2000,sports,NaN,sullivan award,2006-10-13,2006
473785,\'80s LADIES,2,2000,sports,NaN,vanity fair,2021-07-27,2021
247676,WOMEN IN SPORTS,2,2000,sports,NaN,nancy lieberman,2004-06-29,2004
399121,FAMILIAR SOUNDING TRIPLES,2,2000,sports,NaN,stop drop role,2015-11-11,2015


In [55]:
qwc_sql_mod =qwc_sql[qwc_sql.question_modified.apply(lambda x: len(str(x))> 2)]

query = """
select 
*, 
ROW_NUMBER() over (partition by question_modified order by date) as 'repeated'
from
qwc_sql_mod
order by question_modified
"""

answer_repeat = sqldf(query) #.head(50) #.to_csv('art_check1.csv')

answer_repeat


,category,round,clue_value,max_predict,second_max_predict,question_modified,date,year,repeated
0,LET'S PLAY ALGEBRA!,2,1600,science,None,0 0,2009-11-13 00:00:00.000000,2009,1
1,GAMES PEOPLE PLAY,2,1600,sports,None,0 00,2002-04-25 00:00:00.000000,2002,1
2,CHANCE,2,800,miscellaneous,None,0 00,2011-07-13 00:00:00.000000,2011,2
3,GREEN THINGS,1,800,science,None,0 00,2014-01-06 00:00:00.000000,2014,3
4,LET'S GAMBLE,2,1200,sports,None,0 00,2022-05-16 00:00:00.000000,2022,4
...,...,...,...,...,...,...,...,...,...
522898,THE GERMAN UMLAUT,1,600,miscellaneous,None,über,2018-11-26 00:00:00.000000,2018,2
522899,DIËRESIS & ÜMLAUT WÖRDS,2,400,miscellaneous,None,über,2021-10-15 00:00:00.000000,2021,3
522900,IT'S GERMAN FOR...,2,1600,miscellaneous,None,übermensch,2023-04-28 00:00:00.000000,2023,1
522901,WORLD HISTORY,2,1600,history,None,điện biên phủ,2024-07-17 00:00:00.000000,2024,1


In [56]:
repeated = pd.DataFrame(columns=['category', 'average', 'clue count'])

for tag in tags:
    sqldf_tag = answer_repeat[(answer_repeat.max_predict == tag) | (answer_repeat.second_max_predict == tag)]
    
    query = """
    select 
    avg(repeated) as 'avg'
    , count(*) as 'clues tagged'
    from sqldf_tag
    where date >= '2016-01-01'
    """


    print(tag)
    yy = sqldf(query)
    
    row = {'category': tag, 'average': yy['avg'][0], 'clue count': yy['clues tagged'][0]}
    repeated.loc[len(repeated)] = row

repeated

art
business
civics
classical music
fashion
film
food
geography
history
literature
miscellaneous
pop culture
pop music
religion
science
sports
television
theater


,category,average,clue count
0,art,33.137355,2155
1,business,24.444110,3650
2,civics,31.040433,2770
3,classical music,28.002859,1399
4,fashion,19.296477,1022
5,film,20.571821,6920
6,food,23.902411,4355
7,geography,57.738760,18416
8,history,42.195220,21299
9,literature,31.207896,12184


In [57]:
repeated.sort_values('average', ascending=False) # average number of times an answerline in this category has been used before

,category,average,clue count
7,geography,57.738760,18416
8,history,42.195220,21299
17,theater,37.322296,3292
13,religion,33.303734,2973
0,art,33.137355,2155
9,literature,31.207896,12184
2,civics,31.040433,2770
3,classical music,28.002859,1399
14,science,27.442071,15873
1,business,24.444110,3650


In [58]:
qwc_sql['year'] = [i.year for i in qwc_sql['date']]

query = """
select 
qwc_sql.*
, freq.freq
, case when freq.debut_date = qwc_sql.date then 1 else 0 end as debut
from 
qwc_sql
left join (select 
question_modified,
count(*) as freq, 
min(date) as debut_date
from qwc_sql
group by question_modified) freq
on qwc_sql.question_modified = freq.question_modified"""

answer_freq = sqldf(query) #.head(50) #.to_csv('art_check1.csv')

answer_freq

sqldf24 = answer_freq[(answer_freq.date > '2015-01-01')] #& (answer_freq.second_max_predict.isnull())]

query = """
select max_predict, avg(freq), sum(debut), count(*)
from sqldf24
group by max_predict"""

sqldf(query)

,max_predict,avg(freq),sum(debut),count(*)
0,art,37.287918,154,1945
1,business,31.229155,514,3430
2,civics,36.774484,274,1889
3,classical music,37.995301,135,1277
4,fashion,22.636986,135,1022
5,film,35.630861,682,6526
6,food,28.001511,539,3971
7,geography,68.087452,1382,17541
8,history,49.603870,1717,18706
9,literature,44.892268,1049,11705


In [60]:
debut_answer_rate = pd.DataFrame(columns=['category', 'new answerline', 'all clues', 'rate', 'examples'])

for tag in tags:
    sqldf_tag = answer_freq[(answer_freq.max_predict == tag) | (answer_freq.second_max_predict == tag)]
    
    query = """
    select sum(debut), count(*)
    from sqldf_tag
    where date >= '2016-01-01'
    """

    debut_frame = sqldf_tag[(sqldf_tag.debut == 1) & (sqldf_tag.date >= '2020-01-01')]
    debut_list = list(debut_frame.question_modified)

    shuffle(debut_list)
    trigrams = [t for t in zip(*[iter(debut_list)]*6)][:1]

    print(tag)
    print(*trigrams,sep="\n")

    yy = sqldf(query)
    
    row = {'category': tag, 'new answerline': yy['sum(debut)'][0], 'all clues': yy['count(*)'][0], 'rate': yy['sum(debut)'][0] / yy['count(*)'][0]}
    debut_answer_rate.loc[len(debut_answer_rate)] = row

debut_answer_rate

art
('charles mckim', 'agitprop', 'subway car', 'semiquincentennial', 'athens school', 'duck boat')
business
('touchtone dialing', 'express scripts', 'costvolumeprofit', 'barrel barrelhead', 'margin call', 'lillard')
civics
('lds faith mormon church', 'pump breast milk', 'small back', 'marriage story', 'nick kroll', 'chase manhattan')
classical music
('toucan wiccan', 'toot horn', 'j ogden armour', 'reel leer', 'shapenote singing', 'fire rome burned')
fashion
('jean jacket', 'christian siriano', 'brünhilde', 'smith college', 'aviator sunglass', 'outback hat')
film
('nathalie emmanuel', 'mike myers dana carvey', 'zack snyder', 'reefer madness', 'halloweentown', 'ephron')
food
('martha custis washington', 'bay leaf laurel leaf', 'lead lead', 'figgy pudding plum pudding', 'booze cruise', 'beth berth beth')
geography
('university tokyo', 'chain link fence', 'us steel tower', 'clemente', 'ossuary', 'upper new york bay')
history
('earl oxford edward de vere', 'war democrats', 'emperor rome',

,category,new answerline,all clues,rate,examples
0,art,175,2157,0.081131,NaN
1,business,565,3704,0.152538,NaN
2,civics,375,2798,0.134024,NaN
3,classical music,155,1407,0.110163,NaN
4,fashion,137,1024,0.133789,NaN
5,film,741,6999,0.105872,NaN
6,food,607,4369,0.138933,NaN
7,geography,1490,18487,0.080597,NaN
8,history,1993,21418,0.093053,NaN
9,literature,1130,12270,0.092095,NaN


In [691]:
debut_answer_rate.sort_values('rate', ascending=False)

,category,new answerline,all clues,rate,examples
10,miscellaneous,6709,32359,0.207330,NaN
11,pop culture,422,2564,0.164587,NaN
1,business,565,3704,0.152538,NaN
16,television,801,5368,0.149218,NaN
12,pop music,921,6453,0.142724,NaN
6,food,607,4369,0.138933,NaN
2,civics,375,2798,0.134024,NaN
4,fashion,137,1024,0.133789,NaN
15,sports,669,5002,0.133747,NaN
14,science,1993,16016,0.124438,NaN


In [478]:

sqldf24 = answer_freq[(answer_freq.date > '2016-01-01')] #& (answer_freq.second_max_predict.isnull())]

for tag in ['fashion']:
    sqldf_tag = answer_freq[(answer_freq.max_predict == tag) | (answer_freq.second_max_predict == tag)]
    sqldf_tag = sqldf_tag[sqldf_tag.date >= '2016-01-01']
    
    query = """
    select sum(debut), count(*)    from sqldf_tag"""

    yy = sqldf(query)

sqldf_tag[sqldf_tag.debut == 1].sample(10)

,category,round,clue_value,max_predict,second_max_predict,question_modified,date,time_since,is_cat,year,freq,age_years,debut
524134,HAT TRICKS,1,600,fashion,None,outback hat,2025-03-26 00:00:00.000000,128,0,2025,1,0.350445,1
529047,FABRICS & MATERIALS,2,2000,fashion,None,chennai,2025-07-17 00:00:00.000000,15,0,2025,1,0.041068,1
435746,THE FBI's 10 MOST WANTED LIST,1,600,sports,fashion,jail card,2018-07-10 00:00:00.000000,2579,0,2018,1,7.060917,1
428366,"IT'S RAINING ""MEN\""",1,600,miscellaneous,fashion,specimen,2018-01-12 00:00:00.000000,2758,0,2018,1,7.550992,1
483473,FASHION,1,1000,fashion,None,athleisure,2022-04-12 00:00:00.000000,1207,0,2022,1,3.304586,1
527740,CLOTHES TIME,1,800,fashion,None,bodycon,2025-06-18 00:00:00.000000,44,0,2025,1,0.120465,1
478592,FASHION,1,200,fashion,None,little white dress,2021-12-16 00:00:00.000000,1324,0,2021,1,3.624914,1
439176,SCHOOL UNIFORMS,1,800,fashion,None,lands end,2018-11-12 00:00:00.000000,2454,0,2018,1,6.718686,1
484983,"SCORING A ""T—D\""",2,800,miscellaneous,fashion,timid,2022-05-17 00:00:00.000000,1172,0,2022,2,3.003422,1
482975,"FA""SH""ION",2,2000,fashion,None,shantung,2022-03-30 00:00:00.000000,1220,0,2022,1,3.340178,1
